# Preprocessing
The goal of this notebook is to pre-process the input data:
* *cases.txt* containing the number of Covid-19 related confirmed cases and deaths by country
* *population.txt* containing the population and other metadata (% of urban population etc.) by country
* *poultry-production-tonnes.csv* containing the production of poultry by country (and by year) from [ourworlddata](https://ourworldindata.org/)
* *meatconsumption.csv* containing the consumption of meat by category (especially *poultry*) by country and year, source: OECD

In [2]:
import os, re
import pandas as pd

## Processing raw files
### *cases.txt*

In [20]:
lines = []
first = True
with open(os.path.join('data', 'cases.txt'), 'r') as f:
    for line in f:
        line = line.strip().replace(',', '')
        line = re.sub('\t|[\s]{2,}', ',', line)
        if first:
            line = line.replace('Places reporting cases', 'country').lower().replace(' ', '_')
            first = False
        lines.append(line)

with open(os.path.join('data','cases.csv'), 'w+') as f:
    for line in lines:
        f.write(f'{line}\n')

check that the result is readen correctly by *Pandas*:

In [21]:
cases_df = pd.read_csv(os.path.join('data','cases.csv'))

cases_df.head()

,country,sum_of_cases,sum_of_deaths,confirmed_cases_in_the_last_14_days
0,Algeria,39444.0,1391.0,6940.0
1,Angola,1966.0,90.0,802.0
2,Benin,2063.0,39.0,149.0
3,Botswana,1308.0,3.0,504.0
4,Burkina_Faso,1285.0,55.0,129.0


### *population.txt*

In [32]:
lines = []
first = True
with open(os.path.join('data', 'population.txt'), 'r') as f:
    for line in f:
        line = line.strip().replace(',', '').replace(' %', '')
        line = re.sub('^\d+[\t|\s]+', '', line)
        line = re.sub('\t|[\s]{2,}', ',', line)
        if first:
            line = line.replace('.', '_').replace('%', '_perc').replace('#', 'country').replace('CountryPopulation', 'population').replace('YearlyChange', 'yearly_change').replace('UrbanPop', 'urban_pop')
            line = re.sub('\([^\)]*\)', '', line)
            line = line.lower()
            first = False
        lines.append(line)
        
with open(os.path.join('data', 'population.csv'), 'w+') as f:
    for line in lines:
        f.write(f'{line}\n')

In [33]:
population_df = pd.read_csv(os.path.join('data', 'population.csv'))

population_df.head()

,country,population,yearly_change,netchange,density,landarea,migrants,fert_rate,med_age,urban_pop_perc,worldshare
0,China,1439323776,0.39,5540090,153,9388211,-348399,1.7,38,61,18.47
1,India,1380004385,0.99,13586631,464,2973190,-532687,2.2,28,35,17.70
2,United States,331002651,0.59,1937734,36,9147420,954806,1.8,38,83,4.25
3,Indonesia,273523615,1.07,2898047,151,1811570,-98955,2.3,30,56,3.51
4,Pakistan,220892340,2.00,4327022,287,770880,-233379,3.6,23,35,2.83


### *poultry-production-tonnes.csv*

In [50]:
df = pd.read_csv(os.path.join('data', 'poultry-production-tonnes.csv'))
df = df.rename(columns={'Entity': 'country', 'Livestock Primary - Meat, Poultry - 1808 - Production - 5510 - tonnes (tonnes)': 'production'})
df = df.loc[(df['Year'] == 2018) & (df['Code'].notna()),:].reset_index(drop=True)
df.to_csv(os.path.join('data', 'poultry-production.csv'), index=False)
df.head()
                 

,country,Code,Year,production
0,Afghanistan,AFG,2018,28850
1,Albania,ALB,2018,15587
2,Algeria,DZA,2018,294663
3,American Samoa,ASM,2018,25
4,Angola,AGO,2018,27208


### *meatconsumption.csv*

In [20]:
df = pd.read_csv(os.path.join('data', 'meatconsumption.csv'))
df = df.loc[(df['TIME'] == 2018) & (df['SUBJECT'] == 'POULTRY') & (df['MEASURE'] == 'KG_CAP'), ['LOCATION', 'Value']].reset_index(drop=True)
df = df.rename(columns={'LOCATION': 'Code', 'Value': 'consumption'})
df.to_csv(os.path.join('data', 'poultry-consumption.csv'), index=False)
df.head()

,Code,consumption
0,AUS,45.145892
1,CAN,34.668550
2,JPN,14.488577
3,KOR,16.875100
4,MEX,26.610340


## Aggregating countries
The countries in the previous files may have different formatting/name, we have to unify them. The country/code combination in *poultry-production.csv* will serve as pivot so to say

In [125]:
production_df = pd.read_csv(os.path.join('data', 'poultry-production.csv'))
cc = production_df.loc[:,['country', 'Code']]
cc_countries = cc['country'].values.tolist()

In [126]:
cases_df = pd.read_csv(os.path.join('data','cases.csv'))
cases_countries = cases_df['country'].values.tolist()

In [127]:
cases_countries = {c: c.replace('_', ' ') for c in cases_countries}
cases_countries['United_Republic_of_Tanzania'] = 'Tanzania'
cases_countries['Czechia'] = 'Czech Republic'
cases_countries['United_States_of_America'] = 'United States'
cases_countries['Democratic_Republic_of_Congo'] = 'Congo'
cases_countries['Brunei_Darussalam'] = 'Brunei'
cases_countries['Northern_Macedonia'] = 'Macedonia'

In [128]:
cc_countries = {c: c for c in cc_countries}
cc_countries['Swaziland'] = 'Eswatini'
cc_countries["Cote d'Ivoire"] = 'Cote dIvoire'


In [129]:
population_df = pd.read_csv(os.path.join('data', 'population.csv'))
population_countries = population_df['country'].values.tolist()
population_countries = {c: c for c in population_countries}

In [130]:
population_countries['DR Congo'] = 'Congo'
population_countries['North Macedonia'] = 'Macedonia'
population_countries['Czech Republic (Czechia)'] = 'Czech Republic'
population_countries["Côte d'Ivoire"] = 'Cote dIvoire'

In [131]:
valid_countries = set(cases_countries.values()) & set(cc_countries.values()) & set(population_countries.values())

In [132]:
country_df = pd.DataFrame([{'country': c[0], 'country_name': c[1]} for c in cc_countries.items() if c[1] in valid_countries])
production_df = pd.merge(production_df, country_df, on='country').drop('country', 1).rename(columns={'country_name': 'country'})

country_df = pd.DataFrame([{'country': c[0], 'country_name': c[1]} for c in cases_countries.items() if c[1] in valid_countries])
cases_df = pd.merge(cases_df, country_df, on='country').drop('country', 1).rename(columns={'country_name': 'country'})

country_df = pd.DataFrame([{'country': c[0], 'country_name': c[1]} for c in population_countries.items() if c[1] in valid_countries])
population_df = pd.merge(population_df, country_df, on='country').drop('country', 1).rename(columns={'country_name': 'country'})



In [137]:
population_df.drop_duplicates()

,population,yearly_change,netchange,density,landarea,migrants,fert_rate,med_age,urban_pop_perc,worldshare,country
0,1439323776,0.39,5540090,153,9388211,-348399,1.7,38,61,18.47,China
1,1380004385,0.99,13586631,464,2973190,-532687,2.2,28,35,17.70,India
2,331002651,0.59,1937734,36,9147420,954806,1.8,38,83,4.25,United States
3,273523615,1.07,2898047,151,1811570,-98955,2.3,30,56,3.51,Indonesia
4,220892340,2.00,4327022,287,770880,-233379,3.6,23,35,2.83,Pakistan
...,...,...,...,...,...,...,...,...,...,...,...
171,98347,0.62,608,214,460,-200,2.5,34,56,0.00,Seychelles
172,97929,0.84,811,223,440,0,2.0,34,26,0.00,Antigua and Barbuda
173,71986,0.25,178,96,750,N.A.,N.A.,74,0.00,NaN,Dominica
174,62278,-0.36,-228,1246,50,N.A.,N.A.,97,0.00,NaN,Bermuda


In [139]:
df = pd.merge(production_df, cases_df, on='country')
df = pd.merge(df, population_df, on='country')

In [141]:
consumption_df = pd.read_csv(os.path.join('data', 'poultry-consumption.csv'))

In [143]:
df = pd.merge(df, consumption_df, on='Code', how='left')

In [145]:
df.to_csv(os.path.join('data', 'poultry.csv'), index=False)